In [ ]:
import sys
import os
sys.path.append("./../bin/Release")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
goptimum_cec22 = {
    1 : 300, 2: 400, 3: 600, 4: 800, 5:900, 6:1800, 7:2000, 8:2200, 9:2300, 10: 2400, 11:2600, 12: 2700
}
goptimum_cec20 = {
    1 : 100, 2: 1100, 3: 700, 4: 1900, 5:1700, 6:1600, 7:2100, 8:2200, 9:2400, 10: 2500
}
goptimum_cec17 = { i:100*i for i in range(1, 31)}
goptimum_cec20 = np.array(list(goptimum_cec20.values()))
goptimum_cec22 = np.array(list(goptimum_cec22.values()))
goptimum_cec17 = np.array(list(goptimum_cec17.values()))
goptimum_cec17

# CEC 2017

In [ ]:
filename = "./../bin/Release/results_2017_NLSHADE_RSP_10_100000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_ARRDE_10_100000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_LSHADE_10_100000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_jSO_10_100000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2017_NLSHADE_RSP_30_300000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_ARRDE_30_300000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_j2020_30_300000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_LSHADE_30_300000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_jSO_30_300000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0),
    "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2017_NLSHADE_RSP_50_500000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_ARRDE_50_500000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_j2020_50_500000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_LSHADE_50_500000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_jSO_50_500000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
     "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2017_NLSHADE_RSP_100_1000000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_ARRDE_100_1000000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2017_j2020_100_1000000.txt"
#data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_LSHADE_100_1000000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2017_jSO_100_1000000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    #"j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
     "jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0)
    })
df

# CEC 2020

In [ ]:
filename = "./../bin/Release/results_2020_NLSHADE_RSP_10_100000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_ARRDE_10_100000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_jSO_10_100000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2020_NLSHADE_RSP_10_1000000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_ARRDE_10_1000000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_j2020_10_1000000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_LSHADE_10_1000000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_jSO_10_1000000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2020_NLSHADE_RSP_20_200000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_ARRDE_20_200000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2020_j2020_20_200000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0)
    })
df

# CEC 2022


## 10D

In [ ]:
filename = "./../bin/Release/results_2022_NLSHADE_RSP_10_50000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_ARRDE_10_50000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_LSHADE_10_100000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_jSO_10_100000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0), 
    "jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0)
    })
df

In [ ]:
filename = "./../bin/Release/results_2022_NLSHADE_RSP_10_100000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_ARRDE_10_100000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_LSHADE_10_100000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_jSO_10_100000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0),
    })
df

In [ ]:
filename = "./../bin/Release/results_2022_NLSHADE_RSP_10_200000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_ARRDE_10_200000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_j2020_10_200000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_LSHADE_10_200000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_jSO_10_200000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0),
    })
df

In [ ]:
filename = "./../bin/Release/results_2022_NLSHADE_RSP_10_500000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_ARRDE_10_500000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_j2020_10_500000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0)
    })
df

## 20D


In [ ]:
filename = "./../bin/Release/results_2022_NLSHADE_RSP_20_200000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_ARRDE_20_200000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_j2020_20_200000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_LSHADE_20_200000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = "./../bin/Release/results_2022_jSO_20_200000.txt"
data4 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0), 
    "jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0)
    })
df